---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [13]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

In [14]:
nx.__version__

'1.11'

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [3]:
def graph_identification():
    '''
    Each graphs should be different in terms of Clusterring Coefficient (CC), Path Length (PL)
    PA   : very low CC, low PL
    SW_L : high CC, high PL
    SW_H : low CC, low PL
    ''' 
    
    graph_method = []
    
    for i in range(len(P1_Graphs)):
        G = P1_Graphs[i]
        max_deg = max(G.degree().values())/nx.number_of_nodes(G)
        avg_clt = nx.average_clustering(G)
        avg_spath = nx.average_shortest_path_length(G)
        #print(max_deg, avg_clt, avg_spath)
        
        if max_deg >= 0.05:
            graph_method.append('PA')
            #print('PA')
        else:
            if avg_clt >= 0.2:
                graph_method.append('SW_L')
                #print('SW_L')
            else:
                graph_method.append('SW_H')
                #print('SW_H')
    
    return graph_method

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [2]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [6]:
def salary_predictions():
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.metrics import roc_auc_score
    
    df = pd.DataFrame(G.nodes(data=True), columns=['node', 'attribute'])
    df['Department'] = df['attribute'].apply(lambda x: x['Department'])
    df['ManagementSalary'] = df['attribute'].apply(lambda x: x['ManagementSalary'])
    df.drop('attribute', axis=1, inplace=True)
    
    # >>> nx.number_connected_components(G_new)
    # 20
    # >>> len(sorted(nx.connected_components(G_new), key=len)[-1])   #19 components are separated from 1 big connected component
    # 986
    # >>> not_connected_nodes = [next(iter(i)) for i in sorted(nx.connected_components(G_new), key=len)[:-1]]
    
    # delete selfloop (useless information and produce ambiguity in some centrality values)
    G_new = G.copy()
    for i in G_new.selfloop_edges():
        G_new.remove_edge(i[0],i[1])
        
    df['degree'] = list(nx.degree(G_new).values())
    df['degree_cent'] = list(nx.degree_centrality(G_new).values())
    df['closeness_cent'] = list(nx.closeness_centrality(G_new).values())
    df['betweeness_cent'] = list(nx.betweenness_centrality(G_new).values())
    df['page_rank'] = list(nx.pagerank(G_new, alpha=0.9).values())
    hubs_, auths_ = nx.hits(G_new, normalized=True)
    df['hubs'] = list(hubs_.values())
    df['auths'] = list(auths_.values())
    
    df_train = df[~pd.isnull(df['ManagementSalary'])]
    df_test = df[pd.isnull(df['ManagementSalary'])]

    X_columns = ['degree', 'closeness_cent', 'page_rank', 'hubs', 'auths']
    X_train = df[~pd.isnull(df['ManagementSalary'])][X_columns]
    y_train = df[~pd.isnull(df['ManagementSalary'])]['ManagementSalary'].apply(lambda x: int(x))
    X_test = df[pd.isnull(df['ManagementSalary'])][X_columns]
    
    clf = DecisionTreeClassifier(max_depth=5)
    clf.fit(X_train, y_train)
    
    # >>> roc_auc_score(y_train, [i[1] for i in clf.predict_proba(X_train)])
    # 0.93981125573257707
    
    '''
    import matplotlib.pyplot as plt
    import seaborn as sns
    sns.set(style="ticks")

    plt.figure()
    sns.pairplot(df_train[['ManagementSalary', 'degree', 'closeness_cent', 'page_rank', 'hubs', 'auths']],
                 hue='ManagementSalary',
                 vars=['degree', 'closeness_cent', 'page_rank', 'hubs', 'auths'],
                 markers='+')
    plt.show()
    '''
    
    return pd.Series([i[1] for i in clf.predict_proba(X_test)], index=X_test.index)

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [3]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [11]:
def new_connections_predictions():
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.metrics import roc_auc_score
    
    G_new = G.copy()
    for i in G_new.selfloop_edges():
        G_new.remove_edge(i[0],i[1])
        
    df = future_connections.copy()
    
    df['common_neigh'] = [len(list(nx.common_neighbors(G_new, e[0], e[1]))) for e in df.index.values]
    df['jaccard_coef'] = [i[2] for i in list(nx.jaccard_coefficient(G_new, df.index.values))]
    df['rsc_allocation'] = [i[2] for i in list(nx.resource_allocation_index(G_new, df.index.values))]
    df['adamic_adar'] = [i[2] for i in list(nx.adamic_adar_index(G_new, df.index.values))]
    df['pref_attach'] = [i[2] for i in list(nx.preferential_attachment(G_new, df.index.values))]
    df['comm_neighbor'] = [i[2] for i in list(nx.cn_soundarajan_hopcroft(G_new, df.index.values, community='Department'))]
    df['comm_rsc'] = [i[2] for i in list(nx.ra_index_soundarajan_hopcroft(G_new, df.index.values, community='Department'))]
    
    X_columns = ['common_neigh', 'jaccard_coef', 'rsc_allocation', 'adamic_adar',
                 'pref_attach', 'comm_neighbor', 'comm_rsc']
    X_train = df[~pd.isnull(df['Future Connection'])][X_columns]
    y_train = df[~pd.isnull(df['Future Connection'])]['Future Connection'].apply(lambda x: int(x))
    X_test = df[pd.isnull(df['Future Connection'])][X_columns]    

    clf = DecisionTreeClassifier(max_depth=5)
    clf.fit(X_train, y_train)
    
    return pd.Series([i[1] for i in clf.predict_proba(X_test)], index=X_test.index)